## **ANALYSIS_masters_make_PCA**

### GOALS of this script:
* make the PCAs for masters thesis
* with all landraces and elite lines & with downsampled DH lines (31 individuals)
* for all SNPs
* final figure desgin
* **remove the unassigned Unterseer_2016 elite lines**
* **remove the duplicated Unterseer_2016 landrace lines and only use the 3 unique landraces**

> #### **removed the suffix from the pop column since in this PCA**  <br /> 
> #### **- there shouldnt be any duplicates**

### **Import packages**

In [ ]:
import numpy as np
import seaborn as sns
sns.set_style('white')
sns.set_style('ticks')
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import allel
import random
import h5py
import bcolz
import csv
from itertools import chain
import os
from collections import Counter
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from matplotlib.legend_handler import HandlerLine2D

### **Subset the populations from the dataset**

In [ ]:
callset = allel.read_vcf('/Users/kschul38/Documents/tsinfer-project/data/2_processed/europe_maize_dataset_600k/merge_merge_all_chrom_all_edits_with_allel_column_vcf.vcf')

In [ ]:
#callset

In [ ]:
sorted(callset.keys())

In [ ]:
#get the samples in the .vcf
samples= callset["samples"]
#samples

In [ ]:
#total number of the samples inn the vcf
len(samples)

In [ ]:
#put all the samples in a Dataframe
sample_subset = pd.DataFrame(data=samples, columns = ['sample_names'])
#sample_subset

In [ ]:
#sample_subset

#### **Mayer_2020_dhlines** - 941 DH individuals based on three landraces

#### 941 DH individuals based on three landraces - 31 individuals per Pop

In [ ]:
#all Mayer_2020 sample names in the dataframe
mayer_2020_dhlines_samples=sample_subset[sample_subset.sample_names.str.contains("_2020_dh")]
#mayer_2020_dhlines_samples

In [ ]:
#number of all Mayer_2020_dhline individuals
print(len(mayer_2020_dhlines_samples))
#mayer_2020_dhlines_samples

#### *DH_KE*

In [ ]:
#all DH_KE samples in the Mayer_2020_dhlines
df_DH_KE_samples=mayer_2020_dhlines_samples[mayer_2020_dhlines_samples.sample_names.str.contains("DH_KE")]

In [ ]:
#number of all DH_KE individuals
print(len(df_DH_KE_samples))
#df_DH_KE_samples

In [ ]:
#take the first 31 DH_KE individuals 
df_DH_KE_samples_first_31=df_DH_KE_samples.head(31)
print(len(df_DH_KE_samples_first_31))
#df_DH_KE_samples_first_31

#### *DH_LL*

In [ ]:
#all DH_LL samples in the Mayer_2020_dhlines
df_DH_LL_samples=mayer_2020_dhlines_samples[mayer_2020_dhlines_samples.sample_names.str.contains("DH_LL")]

In [ ]:
#number of all DH_LL individuals
print(len(df_DH_LL_samples))
#df_DH_LL_samples

In [ ]:
#take the first 31 DH_LL individuals 
df_DH_LL_samples_first_31=df_DH_LL_samples.head(31)
print(len(df_DH_LL_samples_first_31))
df_DH_LL_samples_first_31

#### *DH_PE*

In [ ]:
#all DH_PE samples in the Mayer_2020_dhlines
df_DH_PE_samples=mayer_2020_dhlines_samples[mayer_2020_dhlines_samples.sample_names.str.contains("DH_PE")]

In [ ]:
#number of all DH_PE individuals 
print(len(df_DH_PE_samples))
#df_DH_PE_samples

In [ ]:
#take the first 31 DH_PE individuals
df_DH_PE_samples_first_31=df_DH_PE_samples.head(31)
print(len(df_DH_PE_samples_first_31))
df_DH_PE_samples_first_31

#### Asign populations to the 31 DH sample subsets

In [ ]:
mayer_2020_DH_landraces_KL=df_DH_KE_samples_first_31.assign(Population="KL DH") #(Population="KL_mayer_2020")
mayer_2020_DH_landraces_KL

In [ ]:
mayer_2020_DH_landraces_LL=df_DH_LL_samples_first_31.assign(Population="LL DH") #(Population="LL_mayer_2020")
mayer_2020_DH_landraces_LL

In [ ]:
mayer_2020_DH_landraces_PE=df_DH_PE_samples_first_31.assign(Population="PE DH") #(Population="PE_mayer_2020")
mayer_2020_DH_landraces_PE

#### **Mayer_2017_landraces** - 952 individuals based on 35 landraces

In [ ]:
#all mayer_2017 sample names
mayer_2017_landraces_samples=sample_subset[sample_subset.sample_names.str.contains("_2017_landraces")]

In [ ]:
#number of all Mayer_2017_landraces
print(len(mayer_2017_landraces_samples))
#mayer_2017_landraces_samples

In [ ]:
mayer_2017_landraces_samples

In [ ]:
#add the populations to the individuals
for index, row in mayer_2017_landraces_samples.iterrows():
    #print(row['sample_names'])
    mayer_string=row['sample_names']#take the sample names row
    #print(mayer_string) 
    mayer_substring=mayer_string[0:2] #take the first two letters of the sample names
    #print(mayer_substring)
    mayer_2017_landraces_samples.at[index,'Population'] = mayer_substring #append the first two letters as the pop name 

In [ ]:
mayer_2017_landraces_samples

In [ ]:
#get all the unique population that you have assigned and check the number 
populations_mayer = mayer_2017_landraces_samples.Population.unique()
print(len(populations_mayer))
print(populations_mayer)

In [ ]:
#get the number of the individuals per pop
#Counter(mayer_2017_landraces_samples.Population)

#### **Unterseer_2016_landraces** - 906 individuals based on 38 european landraces

In [ ]:
#all Unterseer_2016_landraces sample names
unterseer_2016_landraces_samples=sample_subset[sample_subset.sample_names.str.contains("_2016_landraces")]
#unterseer_2016_landraces_samples

In [ ]:
#number of all Unterseer_2016_landraces samples
print(len(unterseer_2016_landraces_samples))
#unterseer_2016_landraces_samples

In [ ]:
#add the populations to all the individuals 
for index, row in unterseer_2016_landraces_samples.iterrows():
    #print(row['sample_names'])
    unterseer_string=row['sample_names'] #take the sample names row
    #print(unterseer_string) 
    unterseer_substring=unterseer_string[0:2] #take the first two letters of the sample names
    #print(unterseer_substring)
    unterseer_2016_landraces_samples.at[index,'Population'] = unterseer_substring #append the first two letters as the pop name

In [ ]:
unterseer_2016_landraces_samples

In [ ]:
#get all the unique population that you have assigned and check the number 
populations_unterseer = unterseer_2016_landraces_samples.Population.unique()
print(len(populations_unterseer))
print(populations_unterseer)

In [ ]:
#get the number of the individuals per pop
#Counter(unterseer_2016_landraces_samples.Population)

#### **Select the three unique populations from the Unterseer_2016 landraces**

In [ ]:
unique_unterseer_2016_landraces_samples=unterseer_2016_landraces_samples.loc[unterseer_2016_landraces_samples['Population'].isin(['AL','SO','FL'])]

In [ ]:
unique_unterseer_2016_landraces_samples

In [ ]:
#get all the unique population that you have assigned and check the number 
print(len(unique_unterseer_2016_landraces_samples))
print(pd.unique(unique_unterseer_2016_landraces_samples['Population']))

In [ ]:
#get the number of the individuals per pop
Counter(unique_unterseer_2016_landraces_samples.Population)

#### **Unterseer_2016_elite** - 155 individuals based on 155 maize elite lines

In [ ]:
#all Unterseer_2016_elite sample names
unterseer_2016_elite_sample=sample_subset[sample_subset.sample_names.str.contains("_2016_elite")]
#unterseer_2016_elite_sample

In [ ]:
unterseer_2016_elite_sample

In [ ]:
sample_subset[sample_subset.sample_names.str.contains("Il14H")]

In [ ]:
sample_subset[sample_subset.sample_names.str.contains("W64A")]

#### Make flint and dent lists with the samples to include in the PCA

In [ ]:
#info from Unterseer_2016 supplement
dent_list=["A632","B14","B37","B73","B97","CM105","CO125","CR1Ht","D06","D09","DJ7","EC169","F252","F288","F353","F618","F748","F838","F922",
"F98902","Fv113","Fv230","Fv310","K55","Ky21","LH119","LH132","LH149","LH160","LH202","LH38","LH39","LH51","LH54","LH59","LH61",
"LH65","LH74","LH93","M162W","M37W","MBS847","ML606","MO17","Mo18W","Ms71","OH43","Oh7B","PA91","PH207",
"PHG29","PHG50","PHG72","PHH93","PHJ90","PHK29","PHK42","PHN82","PHP02","PHR25","PHW17","PHW52","UH250","UH304","VA85","W117","W22",
"W401","W64A","WF9","X740"]

In [ ]:
len(dent_list)

In [ ]:
#info from Unterseer_2016 supplement 
flint_list=["CH10","CO255","D102","D118","D141","D142","D143","D152","D171","D503","DK105","EC49A","EM1349",
"EP1","EP42","EP44","EZ5","F03802","F2","F283","F4","F41","F471","F48","F564","F64","F7","F7012","F902","FC13",
"FC1571","FC23","FC24","FC25","FC26","FF0721H.7","FV1","FV10","FV11","FV131","FV226","FV268","FV286","FV324",
"FV65","FV66","FV69A","FV70","FV79","FV83","FV85","Il14H","Lo11","Lo32","P39","UH006","UH007","UH009",
"X1G0895.DH101","X1G0895.DH104","X1G0895.DH109","X1G0896.DH102","X1G0896.DH116","X1G0896.DH123",
"X1G0896.DH212","X1G0897.DH102","X1G0897.DH135","X1G0897.DH203"]

In [ ]:
len(flint_list)

#### Remove the elite lines that were not genotyped with the 600k array from the lists

*Dent lines*

In [ ]:
dent_list.remove("W64A")

In [ ]:
len(dent_list)

*Flint lines*

In [ ]:
flint_list.remove("EZ5")

In [ ]:
flint_list.remove("F64")

In [ ]:
len(flint_list)

In [ ]:
#number matches with the one given in the paper 
70+66

#### Assign the right population to the Unterseer_2016_elite dataframe

In [ ]:
#append the germplasm pool assignment
for index, row in unterseer_2016_elite_sample.iterrows():
    #print(row['sample_names'])
    element=row['sample_names']
    sub_element=element.partition("_")[0]
    #print(sub_element)
    if sub_element in dent_list: 
        unterseer_2016_elite_sample.at[index,'Population'] = "dent"
    if sub_element in flint_list:
        unterseer_2016_elite_sample.at[index,'Population'] = "flint"
    if sub_element not in dent_list and sub_element not in flint_list:
        unterseer_2016_elite_sample.at[index,'Population'] = "-"

In [ ]:
unterseer_2016_elite_sample

In [ ]:
Counter(unterseer_2016_elite_sample["Population"])

#### Make the subset out of the large dataframe

In [ ]:
subset_slash=unterseer_2016_elite_sample.loc[unterseer_2016_elite_sample['Population'] == "-"]
subset_slash

In [ ]:
len(subset_slash)

In [ ]:
len(subset_slash)

In [ ]:
subset_dent=unterseer_2016_elite_sample.loc[unterseer_2016_elite_sample['Population'] == "dent"]
subset_dent

In [ ]:
subset_flint=unterseer_2016_elite_sample.loc[unterseer_2016_elite_sample['Population'] == "flint"]
subset_flint

In [ ]:
#check if the number of assignments equals 136 and the total 155
print(len(subset_slash))
print(len(subset_dent))
print(len(subset_flint))

In [ ]:
19+70+66

In [ ]:
#check the assignment of specific columns by conten
#unterseer_2016_elite_sample.loc[unterseer_2016_elite_sample["F2"]]
#check the assignment of specific column by index
#unterseer_2016_elite_sample.loc[38]

#### Combine flint and dent subsets

In [ ]:
flint_and_dent_unterseer_2016_elite_sample=unterseer_2016_elite_sample.loc[unterseer_2016_elite_sample['Population'].isin(['dent','flint'])]

In [ ]:
#flint_and_dent_unterseer_2016_elite_sample

In [ ]:
flint_and_dent_unterseer_2016_elite_sample[flint_and_dent_unterseer_2016_elite_sample["Population"]=="dent"]

In [ ]:
flint_and_dent_unterseer_2016_elite_sample[flint_and_dent_unterseer_2016_elite_sample["Population"]=="flint"]

*Write the final lists to file*

textfile = open("subset_dent_list.txt", "w")
for element in subset_dent_list:
    textfile.write(element + "\n")
textfile.close()

textfile = open("subset_flint_list.txt", "w")
for element in subset_flint_list:
    textfile.write(element + "\n")
textfile.close()

#### Add all individual lists into one list

> ###### **Unterseer_2016_elite - flint&dent** 
> ###### **Unterseer_2016 landrace lines - 3 unique lines [AL,FL,SO]** <br /> 
> ###### **Mayer_2020 DH lines - 31 individuals per DH** <br />
> ###### **Mayer_2017_landrace - all individuals from the 35 landraces** <br />

#### Unterseer_2016_elite - flint&dent

In [ ]:
#flint_and_dent_unterseer_2016_elite_sample

In [ ]:
print(len(flint_and_dent_unterseer_2016_elite_sample))

#### Unterseer_2016 landrace lines - 3 unique lines [AL,FL,SO]

In [ ]:
#unique_unterseer_2016_landraces_samples

In [ ]:
print(len(unique_unterseer_2016_landraces_samples))

#### Mayer_2020 DH lines - 31 individuals per DH

In [ ]:
#mayer_2020_DH_landraces_KL

In [ ]:
#mayer_2020_DH_landraces_LL

In [ ]:
#mayer_2020_DH_landraces_PE

In [ ]:
print(len(mayer_2020_DH_landraces_KL))
print(len(mayer_2020_DH_landraces_LL))
print(len(mayer_2020_DH_landraces_PE))

#### Mayer_2017_landrace - all individuals from the 35 landraces

In [ ]:
#mayer_2017_landraces_samples

In [ ]:
print(len(mayer_2017_landraces_samples))

#### Combine the subset of samples

In [ ]:
#combine the subset of samples
filtered_subset_samples_landraces_and_Dh_and_elite=pd.concat([flint_and_dent_unterseer_2016_elite_sample,unique_unterseer_2016_landraces_samples,mayer_2020_DH_landraces_KL,mayer_2020_DH_landraces_LL,mayer_2020_DH_landraces_PE,mayer_2017_landraces_samples])

In [ ]:
filtered_subset_samples_landraces_and_Dh_and_elite

In [ ]:
#check the composition of the samples for the PCA
#Counter(filtered_subset_samples_landraces_and_Dh_and_elite["Population"])

In [ ]:
#number of populations included
print(len(filtered_subset_samples_landraces_and_Dh_and_elite.Population.unique()))

In [ ]:
#summarize the lists to check the lenghts 
136+72+31+31+31+952

### **Load the subset .vcf for the 1253 individuals**

In [ ]:
#flatten into 1 D array
filtered_subset_samples_landraces_and_Dh_and_elite_array=filtered_subset_samples_landraces_and_Dh_and_elite["sample_names"].to_numpy().flatten()
filtered_subset_samples_landraces_and_Dh_and_elite_array

In [ ]:
len(filtered_subset_samples_landraces_and_Dh_and_elite_array)

### **Load the subset .vcf for the 2016 individuals**

In [ ]:
#flatten into 1 D array
#subset_samples_landraces_and_DH_and_elite_array=subset_samples_landraces_and_DH_and_elite["sample_names"].to_numpy().flatten()
#subset_samples_landraces_and_DH_and_elite_array

In [ ]:
#len(subset_samples_landraces_and_DH_and_elite_array)

### **Read in the subsets of the samples**

**1253 lines**

In [ ]:
#read in the subset of the samples
loc_samples = allel.read_vcf('merge_merge_all_chrom_all_edits_with_allel_column_from_array_600k_for_masters_vcf.vcf', samples= filtered_subset_samples_landraces_and_Dh_and_elite_array)  

**2106 lines**

In [ ]:
#read in the subset of the samples
#loc_samples = allel.read_vcf('/Users/kschul38/Documents/tsinfer-project/data/2_processed/europe_maize_dataset_600k/merge_merge_all_chrom_all_edits_with_allel_column_vcf.vcf', samples= subset_samples_landraces_and_DH_and_elite_array)  

In [ ]:
sorted(loc_samples.keys())

In [ ]:
#check the number of the samples in the subset .vcf
loc_samples_check= loc_samples["samples"]
print(len(loc_samples_check))

In [ ]:
Counter(filtered_subset_samples_landraces_and_Dh_and_elite["sample_names"]==loc_samples_check)

### **Get the genotype data for the 1253 individuals**

In [ ]:
genotype_array = allel.GenotypeArray(loc_samples['calldata/GT'])
genotype_array

In [ ]:
#check if the genotype array contains the right encoding
check_genotype_array=pd.unique(np.asarray(genotype_array).flatten())
check_genotype_array
#Where 0 is the reference allele
#1 is the first alternate allele
#2 is the second alternate allele
#-1 (or any other negative integer) is a missing allele call

#Thought the -1 was a problem but http://alimanfoo.github.io/2017/06/14/read-vcf.html states:
#By default scikit-allel uses -1 to indicate a missing value for any array with a signed integer data type.

### **Count alleles at each variant**

In [ ]:
#Each integer within the array corresponds to an allele index
allelecountsarray = genotype_array.count_alleles()[:]
allelecountsarray

### **Remove singletons and multiallelic SNPs**

In [ ]:
#number multiallelic SNPs 
np.count_nonzero(allelecountsarray.max_allele() > 1)

In [ ]:
#number of biallelic SNPs
np.count_nonzero(allelecountsarray.max_allele() == 1)

In [ ]:
#number of singletons
np.count_nonzero(allelecountsarray.max_allele() == 0)

In [ ]:
#number biallelic singletons
np.count_nonzero((allelecountsarray.max_allele() == 1) & allelecountsarray.is_singleton(1))

In [ ]:
#total number of SNPs removed
458+182

In [ ]:
Counter((allelecountsarray[:, :2].min(axis=1) > 1))

In [ ]:
#apply flitering, remove singeltons and multiallelic SNPs
flt = (allelecountsarray.max_allele() == 1) & (allelecountsarray[:, :2].min(axis=1) > 1)
genotype_array_filtered = genotype_array.compress(flt, axis=0)
genotype_array_filtered

In [ ]:
#check if the genotype array still contains the right encoding
check_genotype_array_filtered=pd.unique(np.asarray(genotype_array_filtered).flatten())
check_genotype_array_filtered

### **Transform the genotype data into a 2-dimensional matrix**

In [ ]:
#Transform each genotype call into the number of non-reference alleles
number_of_non_reference_alleles = genotype_array_filtered.to_n_alt()
number_of_non_reference_alleles

In [ ]:
#check if the array still contains the right encoding
pd.unique(number_of_non_reference_alleles.flatten())

In [ ]:
#check correct number of SNPs
print(len(number_of_non_reference_alleles))

### **Subset the number of SNPs**

In [ ]:
#n = 20000 #Number of SNPs to choose randomly
#vidx = np.random.choice(number_of_non_reference_alleles.shape[0], n, replace=False)
#vidx.sort()
#number_of_non_reference_alleles_random = number_of_non_reference_alleles.take(vidx, axis=0)
#number_of_non_reference_alleles_random

In [ ]:
#check correct number of SNPs
#print(len(number_of_non_reference_alleles_random))

In [ ]:
#check if the array still contains the right encoding
#pd.unique(number_of_non_reference_alleles_random.flatten())

### **Linkage Disequilibrium pruning**

In [ ]:
#Plotting Linkage Disequilibrium - defining the plotting 
def plot_ld(number_of_non_reference_alleles_random, title):
    m = allel.rogers_huff_r(number_of_non_reference_alleles_random) ** 2
    fig = allel.plot_pairwise_ld(m)
    fig.set_title(title)
    fig.savefig("PCA_LD_before_pruning.pdf",bbox_inches='tight') #save figure as pdf 

#### **Plotting LD before LD pruning** - subset 20.000 SNPs

#plot LD for 20.000 SNPs before pruning
#plot_ld(number_of_non_reference_alleles_random, 'Figure 1A. Pairwise LD for random subset of 20.000 SNPs - not pruned')
#dots show indicate pairs of SNPs where genotypes are correlated.

#### **Plotting LD before LD pruning** - all SNPs

#plot LD for all SNPs before pruning
plot_ld(number_of_non_reference_alleles, 'Figure 1A. Pairwise LD for all SNPs - not pruned')
#dots show indicate pairs of SNPs where genotypes are correlated.

### **Removing correlated features (LD)**

In [ ]:
#Removing correlated features (LD)
def ld_prune(gn, size, step, threshold=.1, n_iter=1):
    for i in range(n_iter):
        loc_unlinked = allel.locate_unlinked(gn, size=size, step=step, threshold=threshold)
        n = np.count_nonzero(loc_unlinked)
        n_remove = gn.shape[0] - n
        print('iteration', i+1, 'retaining', n, 'removing', n_remove, 'variants')
        gn = gn.compress(loc_unlinked, axis=0)
    return gn

pruned_subset = ld_prune(number_of_non_reference_alleles_random, size=500, step=200, threshold=.1, n_iter=5)

In [ ]:
pruned_all = ld_prune(number_of_non_reference_alleles, size=500, step=200, threshold=.1, n_iter=5)

#### **Plotting LD after LD pruning** - subset 20.000 SNPs

#plot LD for all SNPs in the array before pruning
#plot_ld(pruned_subset, 'Figure 1. Pairwise LD for random subset of 20.000 SNPs - pruned')
#dots show indicate pairs of SNPs where genotypes are correlated.

#### **Plotting LD after LD pruning** - all SNPs

#plot LD for all SNPs in the array before pruning
#plot_ld(pruned_all, 'Figure 1. Pairwise LD for all SNPs - pruned')
#dots show indicate pairs of SNPs where genotypes are correlated.

pruned_subset=pruned_subset[:]
pruned_subset

In [ ]:
pruned_all=pruned_all[:]
pruned_all

##check if the array still contains the right encoding
pd.unique(pruned_subset.flatten())

In [ ]:
##check if the array still contains the right encoding
pd.unique(pruned_all.flatten())

### **Make the PCA**

In [ ]:
#check the populations included
populations = filtered_subset_samples_landraces_and_Dh_and_elite.Population.unique()
print(populations)
print(len(populations))

#### **Make PCA** - 20.000 SNP subset

coords1_subset, model1_subset = allel.pca(pruned_subset, n_components=10, scaler='patterson', ploidy=2)

#### **Make PCA** - all SNPs

In [ ]:
coords1_all, model1_all = allel.pca(pruned_all, n_components=10, scaler='patterson', ploidy=2)

#### **Extract the Principal components** - 20.000 SNP subset

pca1_subset=coords1_subset[:,0]

pca2_subset=coords1_subset[:,1]

pca3_subset=coords1_subset[:,2]

#### **Extract the Principal components** - all SNPs

In [ ]:
pca1_all=coords1_all[:,0]

In [ ]:
pca2_all=coords1_all[:,1]

In [ ]:
pca3_all=coords1_all[:,2]

#### **Make PCA Table for each point** - 20.000 SNP subset

PCA_table_SNP_subset=filtered_subset_samples_landraces_and_Dh_and_elite

PCA_table_SNP_subset["pc1"]=pca1_subset
PCA_table_SNP_subset["pc2"]=pca2_subset
PCA_table_SNP_subset["pc3"]=pca3_subset

#table with the name, Population and the first three Principal Components
PCA_table_SNP_subset

#### **Make PCA Table for each point** - all SNPs

In [ ]:
PCA_table_SNP_all=filtered_subset_samples_landraces_and_Dh_and_elite

In [ ]:
PCA_table_SNP_all["pc1"]=pca1_all
PCA_table_SNP_all["pc2"]=pca2_all
PCA_table_SNP_all["pc3"]=pca3_all

### **Load the metadata file**

In [ ]:
meta_PCA=pd.read_csv('metadata_for_PCA.csv') 
meta_PCA

In [ ]:
#set abbreviation as the index
meta_PCA=meta_PCA.set_index('Abb.')
meta_PCA

In [ ]:
#make dictionary of the metadata
meta_PCA_dict=meta_PCA.to_dict('index')
#meta_PCA_dict

In [ ]:
len(meta_PCA_dict)

### **Plot the PCA**

In [ ]:
def plot_pca_coords(coords, model, pc1, pc2, ax, sample_population):
    sns.despine(ax=ax, offset=5)
    x = coords[:, pc1]
    y = coords[:, pc2]
    #print(coords)
    for pop in populations:
        flt = (sample_population == pop)
        #dent, flint & - 
        if pop in meta_PCA_dict.keys(): # go through the dictionary meta_PCA_dict
            #print(pop)
            popstr=str(pop)
            poptext="$\\bf{}$".format(popstr) # make population name to string
            hex_col=meta_PCA_dict.get(pop).get("hex_col") #get the pop dictionary entry and get the value that goes with the "hex_col" key
            shape=meta_PCA_dict.get(pop).get("shape_PCA") #get the pop dictionary entry and get the value that goes with the "shape" key
            full_name_duplicate=meta_PCA_dict.get(pop).get("Full name duplicate") #get the pop dictionary entry and get the value that goes with the "Full name duplicate" key
            #print(full_name_duplicate)
            ax.plot(x[flt], y[flt], marker=shape, linestyle=' ', color=hex_col, #for plotting use the shape and hex_col form the dict
            label=full_name_duplicate + ' ' + "[{}]".format(poptext), markersize=5, mec='k', mew=.5) #markeredgecolor or mec	color markeredgewidth or mew	float
                  
    ax.set_xlabel('PC%s (%.1f%%)' % (pc1+1, model.explained_variance_ratio_[pc1]*100)) # x-axis label, add +1 for correct labeling and *100 to express as percent  
    ax.set_ylabel('PC%s (%.1f%%)' % (pc2+1, model.explained_variance_ratio_[pc2]*100)) # y-axis label, add +1 for correct labeling and *100 to express as percent 

In [ ]:
def fig_pca(coords, model, title, sample_population=None):
    if sample_population is None:
        sample_population = filtered_subset_samples_landraces_and_Dh_and_elite.Population.values
    # plot coords for PCs 1 vs 2, 3 vs 4
    fig = plt.figure(figsize=(26,13 )) #figure size
    ax = fig.add_subplot(1, 2, 1) # add subplot on location on grid
    plot_pca_coords(coords, model, 0, 1, ax, sample_population) #take the PC1 (0) and PC2 (1)
    #ax = fig.add_subplot(1, 2, 2) # add subplot on location on grid
    #plot_pca_coords(coords, model, 1, 2, ax, sample_population) #take the PC2 (1) and PC3 (2)
    handles, labels = ax.get_legend_handles_labels() # get the labels form the dict 
    #print(labels)
    labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0])) # sort labels 
    #print(labels)
    #my_handler = HandlerLine2D(numpoints=3)
    #legend_elements = [Line2D([0], [0], marker='X', color='w', label='European landraces', markeredgecolor="black", markersize=15), #set the specifics for the maize group table
                      #Line2D([0], [0], marker='d', color='w', label='Unterseer landraces', markeredgecolor="black", markersize=15), #set the specifics for the maize group table
                      #Line2D([0], [0], marker='^', color='w', label='DH mother lines', markeredgecolor='black', markersize=15), #set the specifics for the maize group table
                      #Line2D([0], [0], marker='s', color='w', label='DH lines', markeredgecolor='black', markersize=15), #set the specifics for the maize group table
                      #Line2D([0], [0], marker='p', color='w', label='elite breeding lines', markeredgecolor='black', markersize=15)] #set the specifics for the maize group table
    #leg1 = ax.legend(handles=legend_elements,bbox_to_anchor=(1.1,1), loc='upper left', labelspacing = 1.4, title= "$\\bf{Maize\ groups}$",fontsize="large", title_fontsize="x-large") #put the specifics into the box, place the box and name the box 
    #leg2 = ax.legend(handles,labels, bbox_to_anchor=(1.5,1), loc='upper left', ncol=2,title= "$\\bf{Populations}$", fontsize="large",title_fontsize="x-large", markerscale=2.5,labelspacing = 1.2) #get the specifics for the maize pops, put into the box, place and name the box 
    #ax.add_artist(leg1)
    fig.suptitle(title, fontsize="xx-large",fontweight='bold', y=1.02) #add title 
    #fig.tight_layout()


    #fig.savefig("PCA.pdf",bbox_inches='tight') #save figure as pdf 

#plot the figure
fig_pca(coords1_subset, model1_subset, 'European maize PCA')

In [ ]:
#plot the figure
fig_pca(coords1_all, model1_all, 'European maize PCA')

#### **Filter PCA Table to find groupings** - 20.000 SNPs subset

pd.unique(PCA_table_SNP_subset[(PCA_table_SNP_subset.pc1>0)&(PCA_table_SNP_subset.pc2>-20)].Population)

pd.unique(PCA_table_SNP_subset[(PCA_table_SNP_subset.pc1<0)&(PCA_table_SNP_subset.pc2>-20)].Population)

pd.unique(PCA_table_SNP_subset[(PCA_table_SNP_subset.pc1>0)&(PCA_table_SNP_subset.pc2>-0)].Population)

pd.unique(PCA_table_SNP_subset[(PCA_table_SNP_subset.pc1<0)&(PCA_table_SNP_subset.pc2>-40)].Population)

pd.unique(PCA_table_SNP_subset[(PCA_table_SNP_subset.pc1<0)&(PCA_table_SNP_subset.pc2<-40)].Population)

#### **Filter PCA Table to find groupings** - all SNPs

In [ ]:
#dent like landraces
pd.unique(PCA_table_SNP_all[(PCA_table_SNP_all.pc1<0)&(PCA_table_SNP_all.pc2<-90)].Population)

In [ ]:
#Northern european cluster filter 1
pd.unique(PCA_table_SNP_all[(PCA_table_SNP_all.pc1>-10)&(PCA_table_SNP_all.pc2>-70)&(PCA_table_SNP_all.pc2<0)].Population)

In [ ]:
#Northern european cluster filter 2
pd.unique(PCA_table_SNP_all[(PCA_table_SNP_all.pc1>50)&(PCA_table_SNP_all.pc2>0)].Population)

In [ ]:
#Southern european cluster
pd.unique(PCA_table_SNP_all[(PCA_table_SNP_all.pc1<0)&(PCA_table_SNP_all.pc2>0)].Population)

In [ ]:
pd.unique(PCA_table_SNP_all[(PCA_table_SNP_all.pc1<-90)&(PCA_table_SNP_all.pc2<0)].Population)